In [ ]:
#!pip3 install tensorflow==2.0
import numpy as np
import pandas as pd


In [ ]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP+OCR_task/imdb_small.csv')
data_test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP+OCR_task/test.csv')

In [ ]:
data.head()

,review,sentiment
0,I have to differ from the other comments poste...,negative
1,I saw this movie with low expectations and was...,negative
2,Taran Adarsh a reputed critic praised such a d...,negative
3,When I first heard that the subject matter for...,positive
4,"With the release of Peter Jackson's famed ""Lor...",positive


In [ ]:
text=data_test.columns[0]

In [ ]:
data_test.rename(columns = {text:'review'}, inplace = True)

In [ ]:
data_temp=pd.DataFrame([text],columns=['review'])

In [ ]:
data_test=pd.concat([data_temp,data_test],axis=0,ignore_index=True)

In [ ]:
from sklearn import preprocessing 
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
data['sentiment']=label_encoder.fit_transform(data['sentiment']) 

In [ ]:
data.head()

,review,sentiment
0,I have to differ from the other comments poste...,0
1,I saw this movie with low expectations and was...,0
2,Taran Adarsh a reputed critic praised such a d...,0
3,When I first heard that the subject matter for...,1
4,"With the release of Peter Jackson's famed ""Lor...",1


In [ ]:
#Equal number of positive and negative labels
data['sentiment'].value_counts()

1    2500
0    2500
Name: sentiment, dtype: int64

In [ ]:
y=data['sentiment']

In [ ]:
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps=PorterStemmer()
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import spacy
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup

import unicodedata

!pip install contractions


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 245kB 3.6MB/s 
     |████████████████████████████████| 317kB 20.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81693 sha256=90ad64d25de4a68f4fb7dc0a42f38bf18de1a5f8c8484e0a845cb7a11eaf7edc
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
#@title  CONTRACTIONS

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
#We remove no and not from stopwords
#Beacuse they carry strong emotions
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [ ]:
#Preprocesiing (Cleaning) the data

def deEmojify(text):
    return text.encode('ascii', 'ignore').decode('ascii')

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [ ]:
#Using the contraction Map:(Took from Kaggle) and expanding them
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [ ]:
#Remoivng unwanted special characters
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [ ]:
#Removing the html tags
from bs4 import BeautifulSoup
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [ ]:
#Simple stemming
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
#Removing stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
#Putting all of them together
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True, deEmojifying=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # strip Emojis
        if deEmojifying:
            doc = deEmojify(doc)    
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [ ]:
data['clean_review']=normalize_corpus(data['review'])
data_test['clean_review']=normalize_corpus(data_test['review'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,y_train=(data['clean_review'],data['sentiment'])
X_test=data_test['clean_review']

In [ ]:
#@title
'''
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=200)
X_train1=cv.fit_transform(X_train).toarray()
X_val1=cv.fit_transform(X_val).toarray()


from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(max_features=200)
X_train1=tf.fit_transform(X_train).toarray()
X_val1=tf.fit_transform(X_val)

from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(X_train1,y_train)
y_pred=model.predict(X_val1)

from sklearn.metrics import f1_score,accuracy_score
scoref1_nb=f1_score(y_pred,y_val)
scoreacc_nb=accuracy_score(y_pred,y_val)

from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train1,y_train)
pred=lr.predict(X_val1)


from sklearn.metrics import f1_score,accuracy_score
scoref1=f1_score(pred,y_val)
scoreacc=accuracy_score(pred,y_val)
'''



In [ ]:
import tensorflow as tf
#tf.enable_eager_execution()
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
!pip install bert-tensorflow
from bert import tokenization


     |████████████████████████████████| 71kB 2.3MB/s 


In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py


In [ ]:
!pip install sentencepiece
!pip install tokenization
!pip install FullTokenizer
import tokenization

     |████████████████████████████████| 1.0MB 2.8MB/s 
ERROR: Could not find a version that satisfies the requirement FullTokenizer (from versions: none)
ERROR: No matching distribution found for FullTokenizer


In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(X_train.values, tokenizer, max_len=160)
test_input = bert_encode(X_test.values, tokenizer, max_len=160)
train_labels = y_train.values


In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=32
)

Epoch 1/3
  2/125 [..............................] - ETA: 1:33:22 - loss: 0.6863 - accuracy: 0.5000

In [ ]:
model.load_weights('model.h5')

In [ ]:
test_pred = model.predict(test_input)
test_pred = test_pred.round().astype(int)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(test_pred,y_val)
score

0.888

In [ ]:
test_inp = bert_encode(X_test.values, tokenizer, max_len=160)
test_pred_f = model.predict(test_inp)
test_pred_f = test_pred_f.round().astype(int)

In [ ]:
preds=pd.DataFrame(test_pred_f,columns=['Sentiment'])

In [ ]:
final=pd.concat([data_test,preds],axis=1,ignore_index=True)

In [ ]:
final=final.rename(columns={0:'review',1:'clean_review',2:'Sentiment'})
final=final.drop(['clean_review'],axis=1)

In [ ]:
for i in range(len(final)):
  if(final['Sentiment'][i]==0):
    final['Sentiment'][i]='Negative'
  if(final['Sentiment'][i]==1):
    final['Sentiment'][i]='Positive'  

In [ ]:
final.to_csv('Predictions.csv')